In [ ]:
# !pip install langchain-community langchain-openai langchain-chroma langchain langchain-huggingface 

In [4]:
from langchain_community.document_loaders import DirectoryLoader,TextLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
from vllm import LLM
import os

2024-07-03 17:08:17,935	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [5]:
path = "./data/"
loader = DirectoryLoader(path, glob="*.txt", loader_cls=TextLoader)
docs = loader.load()

In [6]:
from config import OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [7]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [7]:
from config import GOOGLE_API_KEY
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

In [12]:
def generate_response(retriever, query):

    try:
        # llm = ChatGoogleGenerativeAI(model="gemini-pro")
        llm = LLM(model="meta-llama/Meta-Llama-3-8B-Instruct")
        # llm = ChatOpenAI(model="gpt-4")
        # llm =chat
#         llm = HuggingFacePipeline.from_model_id(
#     model_id="epfl-llm/meditron-7b",
#     task="text-generation",
# )
        template = """
            You are a helpful Medical AI assistant. Answer the question based on the context provided. Respond only if the answer is in the context.
            If the answer is not in the context then respond, Cannot answer the question.
            Stick to the context while answering and be concise.
            context: {context}
            question: {input}
            answer:
            """
        prompt = PromptTemplate.from_template(template)
        combine_docs_chain = create_stuff_documents_chain(llm, prompt)
        retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)
        response = retrieval_chain.invoke({"input": query})
        # print(response["answer"])
        return response["answer"]
    except Exception as e:
        print(f"An error occurred while generating the response: {e}")
        return None

In [13]:
questions = [
"Given the History of Present Illness, What is the relevant past medical history?",
"Given the HPI, chief complaint, and initial ICD diagnosis, Why was the patient admitted to the hospital? What symptoms, medical condition, or other reason caused the patient to come to or be brought into the hospital?",
"Based on the admission medication list, what other existing or ongoing medical conditions did the patient have?",
"Given the history of present illness, admission and discharge diagnosis, What medical conditions or symptoms was this patient treated for in the hospital?",
"given the history of present illness, What was the initial treatment course?",
"Given the History of Present illness, What was the initial diagnostic work up and pertinent results?",
"Given the radiology notes, What radiology, imaging or other studies that were performed and for what inducation? Summarize the impressions from each study. Make a table of Imaging study, indication, and impression."
]

In [14]:
response = generate_response(retriever, questions[0])
print(response)

INFO 07-03 16:59:47 llm_engine.py:161] Initializing an LLM engine (v0.5.0.post1) with config: model='meta-llama/Meta-Llama-3-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=meta-llama/Meta-Llama-3-8B-Instruct)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 07-03 17:00:20 model_runner.py:160] Loading model weights took 14.9595 GB
INFO 07-03 17:00:22 gpu_executor.py:83] # GPU blocks: 27961, # CPU blocks: 2048
INFO 07-03 17:00:23 model_runner.py:889] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-03 17:00:23 model_runner.py:893] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 07-03 17:00:57 model_runner.py:965] Graph capturing finished in 34 secs.
An error occurred while generating the response: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'vllm.entrypoints.llm.LLM'>
None


In [15]:
response = generate_response(retriever, questions[0])
print(response)

INFO 07-03 17:01:04 llm_engine.py:161] Initializing an LLM engine (v0.5.0.post1) with config: model='meta-llama/Meta-Llama-3-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=meta-llama/Meta-Llama-3-8B-Instruct)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


An error occurred while generating the response: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 
None


In [11]:
for question in questions:
    response = generate_response(retriever, question)
    print(question,"\n")
    print(response)
    print("\n*********************")

Given the History of Present Illness, What is the relevant past medical history? 

- Bipolar  
- Hepatitis C  
- HLD  
- Diabetes, steroid-induced  
- Self-reported bullous pemphigoid, but no clinic or laboratory evidence to support this diagnosis
- COPD  
- Cholelithiasis  
- GERD  
- Aortic aneurysm  
- Osteoporosis

*********************
Given the HPI, chief complaint, and initial ICD diagnosis, Why was the patient admitted to the hospital? What symptoms, medical condition, or other reason caused the patient to come to or be brought into the hospital? 

The patient was admitted to the hospital due to leg swelling, abdominal distention, and complaints of oral ulcers. The patient was experiencing significant leg pain and had a history of pemphigoid, a skin condition. The patient also reported that his abdomen was swollen. His medical condition and symptoms, along with a primary discharge diagnosis of cellulitis of the left lower limb, prompted his admission to the hospital.

*********

In [9]:
import transformers

def initialize_llama_model(model_id):
    pipeline = transformers.pipeline(
        "text-generation",
        model=model_id,
        device=0,
    )
    return pipeline

def generate_response(retriever, query, llama_pipeline):
    try:
        template = """
            You are a helpful Medical AI assistant. Answer the question based on the context provided. Respond only if the answer is in the context.
            If the answer is not in the context then respond, Cannot answer the question.
            Stick to the context while answering and be concise.
            context: {context}
            question: {input}
            answer:
            """
        prompt = PromptTemplate.from_template(template)
        combine_docs_chain = create_stuff_documents_chain(llama_pipeline, prompt)
        retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)
        
        # Prepare the messages for the LLaMA model
        messages = [
            {"role": "system", "content": "You are a helpful Medical AI assistant."},
            {"role": "user", "content": query},
        ]
        
        terminators = [
            llama_pipeline.tokenizer.eos_token_id,
            llama_pipeline.tokenizer.convert_tokens_to_ids("")
        ]

        outputs = llama_pipeline(
            messages,
            max_new_tokens=256,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
        )
        
        # Extract the generated text from the output
        generated_text = outputs[0]["generated_text"][-1]
        
        # Assuming the answer is directly in the generated text
        return generated_text
        
    except Exception as e:
        print(f"An error occurred while generating the response: {e}")
        return None

# Initialize the LLaMA model
model_id = "/work/AI-EHR/discharge_me/Notebooks/Dushyant_notebooks/meta-llama/Meta-Llama-3-8B-Instruct"
llama_pipeline = initialize_llama_model(model_id)

# Example usage
query = "What are the symptoms of COVID-19?"
response = generate_response(retriever, query, llama_pipeline)
print(response)

Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.26s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


An error occurred while generating the response: 'NoneType' object cannot be interpreted as an integer
None
